该 ipynb 文件构建了一个基于 transformers 的模型训练框架，用于训练 GPT2 模型。

该框架包含五个部分：

- 模型：定义模型的结构
- 评估器：评估模型的性能
- 训练器：定义模型的训练过程
- 数据处理器：处理数据
- 训练：调用训练器进行训练。

受限于笔记本配置，本地训练较慢，因此迁移至 Google Colab 进行训练。在第五部分添加了 Google 云端硬盘的挂载，将数据集和模型保存至云端硬盘。

## 第一步：定义模型

In [ ]:
import json
import logging
import os
from typing import Dict

import torch
from torch import nn, Tensor
from transformers import GPT2LMHeadModel, GPT2Tokenizer

logging.basicConfig(
    format=logging.BASIC_FORMAT,
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)
logger = logging.getLogger(__name__)


class GPTSingleHead(nn.Module):
    """
    Different from directly using GPT2LMHeadModel, this wraps up GPT2LMHeadModel as well as GPT2Tokenizer
    """

    def __init__(self, model_name_or_path: str, max_seq_length: int = 256, do_lower_case: bool = False,
                 special_words_to_add=None):
        """
        定义了一个名为 GPTSingleHead 的 PyTorch 模型类，用于创建 GPT2 模型
        :param model_name_or_path: 指定要加载或初始化的 GPT2 模型的名称或路径。
        :param max_seq_length: 指定输入序列的最大长度。
        :param do_lower_case: 指定是否将输入文本转换为小写。
        :param special_words_to_add: 一个可选参数，用于指定要添加到 tokenizer 中的特殊词语。如 <python>, <java>
        """
        super(GPTSingleHead, self).__init__()
        self.config_keys = ['max_seq_length', 'do_lower_case']
        self.do_lower_case = do_lower_case
        if max_seq_length > 1024:
            logging.warning(
                "GPT only allows a max_seq_length of 1024. Value will be set to 1024")
            max_seq_length = 1024
        self.max_seq_length = max_seq_length
        self.gpt = GPT2LMHeadModel.from_pretrained(model_name_or_path)
        self.tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path, do_lower_case=do_lower_case)
        if special_words_to_add != None:
            self.add_special_words(special_words_to_add)

        self.bos_token_id = self.tokenizer.bos_token_id
        self.eos_token_id = self.tokenizer.eos_token_id
        # self.pad_token_id=self.tokenizer.pad_token_id

    def tokenize(self, text: str):  # default for cls
        """
        将输入文本转换为 token IDs 的序列。
        首先使用 tokenizer.tokenize 将文本标记化为 token 列表，然后使用 tokenizer.convert_tokens_to_ids 将 token 列表转换为对应的 token IDs。
        :param text: 输入文本
        :return: token IDs 的序列
        """
        return self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(text))

    def add_special_words(self, special_words_to_add):
        """
        添加特殊词语到 tokenizer 中，并调整模型的 token embeddings 大小以适应新的词汇量。
        :param special_words_to_add: 要添加到 tokenizer 中的特殊词语
        :return: None
        """
        orig_num_tokens = len(self.tokenizer)
        num_added_tokens = self.tokenizer.add_special_tokens(special_words_to_add)
        if num_added_tokens > 0:
            self.gpt.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens)

    def forward(self, input: Dict[str, torch.Tensor]):
        """
        定义模型的前向传播逻辑。接收一个名为 input 的字典作为输入，包含键为"input_ids"的输入token IDs。
        使用 self.gpt 模型将 input["input_ids"]作 为输入，并返回损失和 logit（模型的输出）。

        :param input: 一个字典，包含键为"input_ids"的输入token IDs。
        :return:  损失和 logit（模型的输出）
        """
        loss, logits = self.gpt(input["input_ids"], labels=input["input_ids"])[:2]
        return loss, logits

    def get_config_dict(self):
        """
        返回模型的配置字典，该字典包含在初始化函数中定义的配置参数。
        """
        return {key: self.__dict__[key] for key in self.config_keys}

    def padding_features(self, features_dict_list):
        """
        padding features for a batch
        对一个batch的特征进行padding。
        遍历features_dict_list中的每个特征字典，将每个特征的token IDs加入到对应的batch列表中。
        找到batch中最长的输入序列长度max_input_len_this_batch。
        根据max_input_len_this_batch对每个特征的token IDs进行padding，使其长度一致。
        :param features_dict_list: i.e., batch
        :return: padded batch features
        """
        max_input_len_this_batch = 0

        batch_features = {feature_name: [] for feature_name in features_dict_list[0]}
        for feature_dict in features_dict_list:
            for feature_name, feature_ids in feature_dict.items():
                if feature_name == "input_ids" and len(feature_ids) > max_input_len_this_batch:
                    max_input_len_this_batch = len(feature_ids)
                batch_features[feature_name].append(feature_ids)

        padded_batch_features = {feature_name: [] for feature_name in features_dict_list[0]}
        for feature_name, batch_ids in batch_features.items():

            for each_ids in batch_ids:
                padded = each_ids + [self.tokenizer.pad_token_id] * (max_input_len_this_batch - len(each_ids))
                padded_batch_features[feature_name].append(padded)

        for feature_name, ids in padded_batch_features.items():
            padded_batch_features[feature_name] = torch.tensor(ids)

        return padded_batch_features

    def get_embedding_dimension(self) -> int:
        """
        返回模型的嵌入维度。
        """
        return self.gpt.config.hidden_size

    def get_config(self) -> int:
        """
        返回模型的配置。
        """
        return self.gpt.config

    def save(self, output_path: str):
        """
        保存模型的权重、tokenizer和配置字典到指定路径。
        :param output_path: 模型保存路径
        :return: None
        """
        self.gpt.save_pretrained(output_path)
        self.tokenizer.save_pretrained(output_path)
        with open(os.path.join(output_path, 'gpt_sh_config.json'), 'w') as f:
            json.dump(self.get_config_dict(), f, indent=2)

    def reload(self, input_path: str):
        """reload from checkpoint weights"""
        return GPTSingleHead.load(input_path + "/0_GPTSingleHead")

    @staticmethod
    def load(input_path: str):
        if not os.path.isfile(os.path.join(input_path, 'gpt_sh_config.json')):
            raise ValueError("In the model path does not find gpt_sh_config.json file, you may have not trained yet")
        with open(os.path.join(input_path, 'gpt_sh_config.json')) as f:
            config = json.load(f)
        return GPTSingleHead(model_name_or_path=input_path, **config)


class EmptyHeads(nn.Module):
    def __init__(self):
        self.config_keys = []
        super().__init__()

    def forward(self, input: Dict[str, Tensor]):
        return input

    def get_config_dict(self):
        return {key: self.__dict__[key] for key in self.config_keys}

    def save(self, output_path):
        with open(os.path.join(output_path, 'empty_heads_config.json'), 'w') as f:
            json.dump(self.get_config_dict(), f, indent=2)
        torch.save(self.state_dict(), os.path.join(output_path, 'empty_heads.pt'))

    def load_saved(self, input_path):
        self.load_state_dict(torch.load(os.path.join(input_path, '1_EmptyHeads', 'empty_heads.pt')))

    @staticmethod
    def load(input_path, config):
        if not os.path.isfile(os.path.join(input_path, 'empty_heads_config.json')):
            raise ValueError(
                "In the model path does not find empty_heads_config.json file, you may have not trained yet")

        with open(os.path.join(input_path, 'empty_heads_config.json')) as f:
            config = json.load(f)
        model = EmptyHeads()

        if not os.path.isfile(os.path.join(input_path, 'empty_heads.pt')):
            raise ValueError("In the model path does not find state of file, you need to train and get weights first")

        model.load_state_dict(torch.load(os.path.join(input_path, 'empty_heads.pt')))
        return model


## 第二步：定义评估器

In [ ]:
import logging
from typing import List, Dict

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

logging.basicConfig(
    format=logging.BASIC_FORMAT,
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)
logger = logging.getLogger(__name__)


class SingleCLMEvaluator():
    def __init__(self, dataloader: DataLoader = None,
                 data_tag: str = "dev",
                 device: int = None, tokenizer=None, early_stop_on: str = "perplexity"):

        if data_tag not in ["dev", "train", "test"]:
            raise ValueError("data_tag has to be one of dev, train or test")
        assert early_stop_on in ["loss", "perplexity"]
        self.early_stop_on = early_stop_on
        self.dataloader = dataloader
        self.data_tag = data_tag
        self.tokenizer = tokenizer

        self.n_gpu = torch.cuda.device_count()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if device == -1:
            self.n_gpu = 0
            self.device = torch.device("cpu")

    def reset_dataloader(self, dataloader: DataLoader):
        self.dataloader = dataloader

    def reset_logger(self, output_path):
        pass

    def __call__(self, model, collate_fn, output_path: str = None, epoch: int = -1, steps: int = -1,
                 target_names: List[str] = None, do_predict: bool = False) -> Dict[
        str, float]:

        if do_predict and self.tokenizer == None:
            raise ValueError("you are doing predict so need a tokenizer")
        if self.dataloader is None:
            raise ValueError(" need to set dataloader for this evaluator, call reset_dataloader()")

        model.eval()
        if epoch == -1 and steps == -1:
            logger.info(
                f"\nEvaluation the model on {self.data_tag} dataset")
        else:
            logger.info(
                "\nEvaluation the model on " + self.data_tag + " dataset" + f" in epoch {epoch} after {steps} steps:")

        self.dataloader.collate_fn = collate_fn
        total_loss = 0.0
        total_steps = 0

        for step, batch in enumerate(tqdm(self.dataloader, desc="evaluating")):
            input = batch["features"]
            # batch to device
            for feature_name, ids in input.items():
                input[feature_name] = ids.to(self.device)

            with torch.no_grad():
                loss, logits = model(input)
                loss = loss.mean()
                total_loss += loss

            total_steps += 1
        eval_loss = total_loss / total_steps
        eval_results = {"loss": eval_loss}

        perplexity = torch.exp(torch.tensor(eval_loss)).clone().detach()
        eval_results["perplexity"] = perplexity.mean().item()
        return eval_results

## 第三步：定义训练器

In [ ]:
import json
import math
import os
import shutil
import sys
from typing import Type, Dict

import torch
import transformers

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader
from torch.utils import data
from torch.optim.optimizer import Optimizer
from tqdm import trange, tqdm

from dateutil.relativedelta import relativedelta

import random
import numpy as np
import logging

logging.basicConfig(
    format=logging.BASIC_FORMAT,
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

from datetime import datetime

try:
    import wandb

    wandb.ensure_configured()
    if wandb.api.api_key is None:
        _has_wandb = False
        wandb.termwarn("W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.")
    else:
        _has_wandb = False if os.getenv("WANDB_DISABLED") else True
except ImportError:
    _has_wandb = False


def set_seed(seed, n_gpu):
    logger.info(f"   see seed for random, numpy and torch {seed}")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)


def print_model_state_dict(model):
    for param_tensor in model.state_dict():
        logger.info(f"{param_tensor}\t{model.state_dict()[param_tensor].size()}")


def print_optimizer_state_dict(optimizer):
    for var_name in optimizer.state_dict():
        logger.info(f"{var_name}\t{optimizer.state_dict()[var_name]}")


def count_params(model: torch.nn.Module, print_details: bool = False):
    trainable_count = 0
    total_count = 0
    if isinstance(model, torch.nn.Sequential):
        for index in model._modules:
            if print_details:
                print_model_state_dict(model._modules[index])
                logger.info(model._modules[index])
            trainable_count += sum(p.numel() for p in model._modules[index].parameters() if p.requires_grad)
            total_count += sum(p.numel() for p in model._modules[index].parameters())
    else:
        if print_details:
            print_model_state_dict(model)
            logger.info(model)
        total_count = sum(p.numel() for p in model.parameters())
        trainable_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logger.info(f'  Total params: {total_count}')
    logger.info(f'  Trainable params: {trainable_count}')
    logger.info(f'  Non-trainable params: {total_count - trainable_count}')


def batch_to_device(batch, device, keep_label=False):
    features = batch['features']
    if isinstance(features, dict):
        for feature_name in features:
            features[feature_name] = features[feature_name].to(device)
    else:
        for inx in range(len(features)):
            for feature_name in features[inx]:
                features[inx][feature_name] = features[inx][feature_name].to(device)

    label_space = batch['labels']
    if label_space == None:  # for tasks like lm, labels are none.
        return features, None
    if not keep_label:
        labels = {"label_space_" + str(inx): label_space[inx].to(device) if torch.is_tensor(label_space[inx]) else
        label_space[inx] for inx in range(len(label_space))}
    else:
        labels = label_space
    return features, labels


def is_wandb_available():
    return _has_wandb


class CollateFunction():
    def __init__(self, up_model):
        self.up_model = up_model

    def __call__(self, batch):
        if isinstance(batch[0], dict):
            padded_features = self.up_model.padding_features(batch)
            return {'features': padded_features,
                    "labels": None}  # label_ids are in features, this task does not need labels, we set


class ModelTrainer():
    def __init__(self, up_model: nn.Module, down_layer: nn.Module = None, train_dataset=None,
                 dev_dataset=None, dev_evaluator=None,
                 epochs: int = 1,
                 visiable_device: str = "0",
                 scheduler: str = 'warmuplinear',
                 warmup_ratio: float = 0.1,
                 optimizer_class: Type[Optimizer] = transformers.AdamW,
                 optimizer_params: Dict[str, object] = {'lr': 5e-5, 'eps': 1e-6, 'correct_bias': False},
                 weight_decay: float = 0.01,
                 early_stop: int = 20,
                 # 20 evaluation steps without improving on the early_stop_on metric as specified in dev_evaluator
                 evaluation_steps: int = 500,
                 output_path: str = None,
                 save_best_model: bool = True,
                 max_grad_norm: float = 1,
                 fp16: bool = False,
                 accumulation_steps=1,
                 fp16_opt_level: str = 'O1',
                 seed: int = 122,
                 data_loader_shuffle=True,
                 device: str = None,
                 dev_batch_size: int = -1,  # the same as train_batch_size
                 n_gpu: int = None,
                 report_model: bool = True,
                 per_gpu_train_batch_size: int = 8,
                 restore_training: bool = False,
                 local_rank: int = -1,
                 wandb_config=None):
        """
        this trainer is written for training a sequential model that contains an upstream_layer (usually transformers)
        and a downstream_layer (usually task-specific heads like FF, RNN, CNN for encoding the output of upstram_layer)

        :param up_model: transformers like transformers.GPT2LMHeadModel or transformers.BERTModel
        :param down_layer: None if up_model already wraps up with an output encoder such as LMHead in GPT2LMHeadModel, else nn.Module for encoding the output of up_model
        :param train_dataset: train_dataset, it can be either instance of torch.data.Dataset or IterableDataset (defined in data.py)
        :param dev_dataset: dev_dataset, it can be either instance of torch.data.Dataset or IterableDataset
        :param dev_evaluator: dev_evaluator, evaluator on dev_dataset for early stop and performance tracking during training (defined in evaluate.py)
        :param epochs: number of epoches for training
        :param visiable_device: devices chosen to perform training
        :param scheduler: scheduler specially from transformers: see options in self._get_scheduler
        :param warmup_ratio: warmup_ratio ratio for learning rate over total training steps
        :param optimizer_class: transformers.AdamW de byfault
        :param optimizer_params: optimizer params
        :param weight_decay:weight decay
        :param early_stop:early stop steps
        :param evaluation_steps:logging steps
        :param output_path: path to save the checkpoint with the best performance as specified in early_stop_on in dev_evaluator instance
        :param save_best_model:save best checkpoint or the latest checkpoint
        :param max_grad_norm:max grad norm
        :param fp16: fp16 training
        :param accumulation_steps:accumulation steps
        :param fp16_opt_level:fp16 opt level
        :param seed:random seed for reproducibility
        :param data_loader_shuffle:Whether to shuffle data_loader of training dataset and dev dataset after epoch ends
        :param device: device for training, None or gpu for gpu training, cpu for gpu training
        :param dev_batch_size: development batch size, usually larger than training batch size due to no grads calculation and hence less burden on memory
        :param n_gpu: number of gpus for training
        :param report_model:if report model's structure and number of trainable params in logging
        :param per_gpu_train_batch_size: what it means literally
        :param restore_training: if restore training if the training process is interupped due to some accidents
        :param local_rank:for distributed training
        :param wandb_config: wandb logging if not none, else without wandb logging
        """

        self.up_model = up_model
        if down_layer == None:
            # In this example, the upstream_layer already integrate the downstream head (namely, simple LM head as in transformers.GPT2LMHeadModel)
            # EmptyHeads is created here only for placeholder purpose
            down_layer = EmptyHeads()

        self.down_layer = down_layer
        assert output_path != None
        output_path = os.path.join("tmp", output_path)
        # os.makedirs(output_path,exist_ok=True)
        if restore_training:
            if not os.listdir(output_path):
                raise ValueError(f"no checkpoint found in {output_path}")
            else:
                logger.info("   loading embedding weights from saved checkpoint")
                self.up_model = self.up_model.reload(
                    output_path)  # for other transformers (apart from bert), the load_saved function has not been added

                logger.info("   loading downstream weights from saved checkpoint")
                self.down_layer.load_saved(output_path)
                with open(output_path + "/ck_report.json") as f:
                    self.ck_report = json.load(f)

        self.model = torch.nn.Sequential(self.up_model, self.down_layer)

        if is_wandb_available() and wandb_config != None:
            # keep track of model topology and gradients if is_wandb_available and args!=None
            wandb.init(project=wandb_config.wandb_project_name, config=wandb_config, name=wandb_config.wandb_run_name)
            wandb.watch(
                (self.up_model, self.down_layer), log_freq=max(100, evaluation_steps)
            )
        self.wandb_config = wandb_config

        self._restore_training = restore_training
        self.early_stop = early_stop

        self._dev_evaluator = dev_evaluator

        self._evaluation_steps = evaluation_steps
        self._save_best_model = save_best_model
        self._max_grad_norm = max_grad_norm

        os.makedirs(output_path, exist_ok=True)
        if os.listdir(output_path) and not restore_training:
            # out = input(
            #     "Output directory ({}) already exists and is not empty, you wanna remove it before start? (y/n)".format(
            #         output_path))
            # if out == "y":
            #     shutil.rmtree(output_path)
            #     os.makedirs(output_path, exist_ok=True)
            # else:
            #     raise ValueError("Output directory ({}) already exists and is not empty".format(
            #         output_path))
            shutil.rmtree(output_path)
            os.makedirs(output_path, exist_ok=True)

        logFormatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        fileHandler = logging.FileHandler(os.path.join(output_path, "log.out"), mode="a")
        fileHandler.setFormatter(logFormatter)
        logger.addHandler(fileHandler)
        self._dev_evaluator.reset_logger(output_path)

        self.output_path = output_path

        if device is None or device == "cuda":
            if torch.cuda.is_available():
                device = torch.device("cuda")
                n_gpu = 1 if n_gpu == 1 else torch.cuda.device_count()
            else:
                logger.warning("no cuda is found in your machine, now use cpu")
                device = torch.device("cpu")
                n_gpu = 0
        elif device == "cpu":
            device = torch.device("cpu")
            n_gpu = 0
        else:
            raise ValueError("set device to be None, cuda or cpu")
        assert n_gpu <= torch.cuda.device_count()

        logger.info("Use pytorch device: {}, with gpu_number={}".format(device, n_gpu))

        self._train_batch_size = per_gpu_train_batch_size * max(1, n_gpu)
        self._dev_batch_size = dev_batch_size if dev_batch_size != -1 else self._train_batch_size

        if isinstance(train_dataset, data.IterableDataset):
            self._train_dataloader = DataLoader(train_dataset, batch_size=None)
            self._steps_per_epoch = len(self._train_dataloader.dataset)
        else:
            self._train_dataloader = DataLoader(train_dataset, shuffle=data_loader_shuffle,
                                                batch_size=self._train_batch_size)
            self._steps_per_epoch = len(self._train_dataloader)

        if isinstance(dev_dataset, data.IterableDataset):
            dev_dataloader = DataLoader(dev_dataset, batch_size=None)
        else:
            dev_dataloader = DataLoader(dev_dataset, shuffle=data_loader_shuffle, batch_size=self._dev_batch_size)

        if accumulation_steps > 1:
            self._steps_per_epoch = self._steps_per_epoch // accumulation_steps

        self._dev_data = dev_dataset
        self._dev_evaluator.reset_dataloader(dev_dataloader)

        self.collate_fn = CollateFunction(self.up_model)
        # Use customize batching
        self._train_dataloader.collate_fn = self.collate_fn

        self._train_data = train_dataset
        self._per_gpu_train_batch_size = per_gpu_train_batch_size

        set_seed(seed, n_gpu)

        if n_gpu > 1:
            self.model = torch.nn.DataParallel(self.model, device_ids=[int(i) for i in visiable_device.split(',')])
            self.model = self.model.to(f'cuda:{self.model.device_ids[0]}')

        elif n_gpu == 1:
            self.model = self.model.to(device)

        self._device = device
        self._n_gpu = n_gpu

        self._total_train_steps = int(self._steps_per_epoch * epochs)
        self._epochs = epochs

        if report_model:
            count_params(self.model, print_details=True)

        param_optimizer = list(self.model.named_parameters())

        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
             'weight_decay': weight_decay},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        if local_rank != -1:
            self._total_train_steps = self._total_train_steps // torch.distributed.get_world_size()

        self._optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

        warmup_steps = math.ceil(self._total_train_steps * warmup_ratio)  # by default 20% of train data for warm-up
        logger.info(f"   Warmup-steps: {warmup_steps}")

        self._scheduler = self._get_scheduler(self._optimizer, scheduler=scheduler, warmup_steps=warmup_steps,
                                              num_total=self._total_train_steps)

        if fp16:
            try:
                from apex import amp
            except ImportError:
                raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

            model, optimizer = amp.initialize(self.model, self._optimizer, opt_level=fp16_opt_level)
            self.model = model
            self._optimizer = optimizer

        self._fp16 = fp16
        tb_writer = None
        if local_rank in [-1, 0]:
            tb_writer = SummaryWriter()
        self._tb_writer = tb_writer
        self._local_rank = local_rank
        self._best_score = -float("inf")
        self._early_stop_count = 0
        self.last_time = datetime.now()
        self.accumulation_steps = accumulation_steps
        # assert evaluation_steps % accumulation_steps == 0, "evaluation_steps should be divisable by accumulation_steps"

    def _train_epoch(self, epoch: int, global_steps: int):
        epoch_steps = 0
        epoch_loss = 0.0

        self.model.zero_grad()
        # 首先，初始化epoch的步数（epoch_steps）和损失（epoch_loss）为0，并清空模型的梯度（self.model.zero_grad()）。

        for step, data in enumerate(
                tqdm(self._train_dataloader, desc="training", total=self._steps_per_epoch * self.accumulation_steps)
        ):
            # 然后，对训练数据进行迭代，使用enumerate函数遍历数据加载器（self._train_dataloader）。
            # tqdm用于在终端显示进度条，desc参数设置进度条的描述为"training"，
            # total参数设置总的迭代次数为self._steps_per_epoch * self.accumulation_steps。

            self.model.train()
            if data["labels"] != "skip-device":
                input, labels = batch_to_device(data, self._device)
                # add labels to input for training where this step is ignored when inference
                if isinstance(labels, dict):
                    for idx in range(len(input)):
                        input[idx].update(labels)
            else:
                input = data["features"]
            # 在每个步骤中，将模型设置为训练模式（self.model.train()）。
            # 然后，检查数据中是否存在标签（data["labels"] != "skip-device"）。
            # 如果存在标签，则将输入数据（input）和标签数据（labels）移到指定设备（self._device）。
            # 如果标签是一个字典，则将标签添加到输入数据中。
            # 如果数据中没有标签，则将输入数据设置为data["features"]。

            loss_value, _ = self.model(input)

            if self._n_gpu > 1:
                loss_value = loss_value.mean()
            if self.accumulation_steps > 1:
                loss_value = loss_value / self.accumulation_steps
            # 计算模型对输入数据的输出（loss_value）。
            # 如果使用多个GPU进行训练（self._n_gpu > 1），则对损失值进行平均。
            # 如果使用梯度累积（self.accumulation_steps > 1），则将损失值除以累积步数。

            if self._fp16:
                try:
                    from apex import amp
                except ImportError:
                    raise ImportError(
                        "Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
                with amp.scale_loss(loss_value, self._optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(self._optimizer), self._max_grad_norm)
            else:
                loss_value.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self._max_grad_norm)
            epoch_loss += loss_value
            # 根据是否启用混合精度训练（self.fp16），计算并反向传播损失值（loss_value）。
            # 如果启用混合精度训练，使用Apex库（需要安装）对损失值进行缩放（amp.scale_loss）和反向传播。
            # 然后，使用梯度裁剪（torch.nn.utils.clip_grad_norm）限制梯度的大小，以避免梯度爆炸。
            # 最后，将损失值添加到epoch_loss中。

            if (step + 1) % self.accumulation_steps == 0:
                self._optimizer.step()
                self._scheduler.step()
                self.model.zero_grad()

                epoch_steps += 1
                total_global = epoch_steps + global_steps
                # 如果达到了累积步数（self.accumulation_steps），则进行梯度更新。
                # 调用优化器的step方法（self._optimizer.step()）执行梯度更新，并调用学习率调度器的step方法（self._scheduler.step()）更新学习率。
                # 然后清空模型的梯度（self.model.zero_grad()）。增加epoch步数（epoch_steps），计算总的全局步数（total_global）。

                if self._evaluation_steps > 0 and (total_global) % self._evaluation_steps == 0:
                    # 如果设置了评估步数（self._evaluation_steps > 0），并且当前步数是评估步数的倍数，则进行模型评估。

                    dev_loss, eval_scores = self._dev_eval_in_training(epoch, epoch_steps)
                    logger.info("   ***** Evaluation report *****")
                    logger.info(f"  Output path (short): {self.output_path}")
                    logger.info(f"  Early stop on: {self._dev_evaluator.early_stop_on}")
                    logger.info(f"  Early stop count = {self._early_stop_count}/{self.early_stop}")
                    logger.info(
                        f"  Eval steps = {self._evaluation_steps} or (iterations = {self._evaluation_steps * self.accumulation_steps})")
                    logger.info(f"  Best score ({self._dev_evaluator.early_stop_on}) = {self._best_score}")
                    logger.info(f"  Gradient Accumulation steps = {self.accumulation_steps}")

                    logger.info(
                        f"  Num of training examples (actually no. of iterations per epoch for Iterable Dataset)  = {len(self._train_data)}")
                    logger.info(
                        f"  Num of development examples (actually no. of iterations per epoch for Iterable Dataset) = {len(self._dev_data)}")
                    now_time = datetime.now()
                    logger.info(f"  Time spent since last evaluation = {self.time_diff(self.last_time, now_time)}")
                    self.last_time = now_time

                    logger.info(f"  Epoch = {epoch + 1}/{self._epochs}")
                    logger.info(f"  Steps = {total_global}/{self._total_train_steps}")
                    logger.info(
                        f"  Instantaneous batch size per GPU = {self._per_gpu_train_batch_size} and n_gpu = {self._n_gpu} so the input batch size = {self._train_batch_size}")
                    if dev_loss != None:
                        logger.info(f"  dev_loss = {dev_loss:.6f}\t||\t dev_eval_scores = {eval_scores}")
                    else:
                        logger.info(f"  dev_eval_scores = {eval_scores}")
                    # 进行模型评估，并获取开发集的损失（dev_loss）和评估指标（eval_scores）。然后，打印评估报告的各种信息，如输出路径、早停策略、评估步数、最佳评估指标分数等。

                    train_loss = epoch_loss / epoch_steps
                    logger.info(f"  train_loss = {train_loss}")
                    logger.info("\n********************************************")

                    if is_wandb_available() and self.wandb_config != None:
                        if dev_loss != None:
                            wandb.log(
                                {"loss_dev": dev_loss,
                                 f"best_score_for_{self._dev_evaluator.early_stop_on}": self._best_score,
                                 "loss_train": train_loss, "lr": self._scheduler.get_lr()[0]},
                                step=total_global)
                        else:
                            wandb.log({"loss_train": train_loss,
                                       f"best_score_for_{self._dev_evaluator.early_stop_on}": self._best_score,
                                       "lr": self._scheduler.get_lr()[0]},
                                      step=total_global)
                    # 计算平均训练损失（train_loss），并将其记录到日志中。
                    # 如果使用了wandb（Weights & Biases）库，并且配置了wandb_config，则将训练损失、开发集损失、最佳评估指标分数和学习率记录到wandb中。

                    for key, value in eval_scores.items():
                        if is_wandb_available() and self.wandb_config != None:
                            wandb.log({f"eval_{key}_dev": value}, step=total_global)
                        self._tb_writer.add_scalar(f"eval_{key}_dev", value, total_global)

                    self._tb_writer.add_scalar("lr", self._scheduler.get_lr()[0], total_global)
                    if dev_loss != None:
                        self._tb_writer.add_scalar("loss_dev", dev_loss, total_global)

                    self._tb_writer.add_scalar("loss_train", train_loss, total_global)

                    if self._early_stop_count >= self.early_stop:
                        logger.info(
                            f"  Continuous {self.early_stop} evaluation steps without loss reduction, so early stopped...")
                        sys.exit(0)
            # 将评估指标的值记录到TensorBoard中，使用TensorBoard写入器（self._tb_writer）。
            # 同时，将学习率、开发集损失和训练损失记录到TensorBoard中。
            # 达到早停策略中设置的停止次数（self._early_stop_count >= self.early_stop），即开发集的损失在连续的评估步骤中没有减少，就会触发早停策略。
            # 在这种情况下，程序记录日志信息，指示连续的评估步骤没有减少损失，并使用sys.exit(0)退出程序。

        return epoch_loss, epoch_steps

    def train(self):
        if self._restore_training:
            logger.info(f"***** restoring training from the previous checkpoint: {self.ck_report}*****")
        else:
            logger.info("***** Running training *****")
        logger.info(
            f"  Num of training examples (actually iterations per epoch for Iterable Dataset) = {len(self._train_data)}")
        logger.info(f"  Output path (short): {self.output_path}")
        logger.info(
            f"  Steps per Epoch = {self._steps_per_epoch} or iterations per epoch = {self._steps_per_epoch * self.accumulation_steps}")
        logger.info(f"  Num of Epochs = {self._epochs}")
        logger.info(f"  Best score ({self._dev_evaluator.early_stop_on}) = {self._best_score}")
        logger.info(
            f"  Eval every {self._evaluation_steps} steps or every {self._evaluation_steps * self.accumulation_steps} iterations")
        logger.info(f"  Early stop = {self.early_stop}")
        logger.info(f"  Gradient Accumulation steps = {self.accumulation_steps}")

        logger.info(f"  Total optimization steps = {self._total_train_steps}")
        logger.info(
            f"  Instantaneous batch size per GPU = {self._per_gpu_train_batch_size} and n_gpu = {self._n_gpu} so the input batch size = {self._train_batch_size}")
        global_loss = 0.0
        global_steps = 0
        self.last_time = datetime.now()
        for epoch in trange(self._epochs, desc="Epoch"):
            epoch_loss, epoch_steps = self._train_epoch(epoch, global_steps)
            global_loss += epoch_loss
            global_steps += epoch_steps
            logger.info(f"epoch {epoch + 1} ends, {self._epochs - epoch - 1} epoches left")
            logger.info(
                f"\nglobal_average_loss={global_loss / global_steps},global_steps={global_steps} on training set")

        if self._local_rank in [-1, 0]:
            self._tb_writer.close()

    def _dev_eval_in_training(self, epoch, steps):
        return_scores = {}
        if self._dev_evaluator is not None:

            return_scores = self._dev_evaluator(self.model, self.collate_fn,
                                                output_path=self.output_path, epoch=epoch, steps=steps)

            early_stop_on = self._dev_evaluator.early_stop_on

            check_score = -return_scores[early_stop_on] if early_stop_on == "loss" or early_stop_on == "perplexity" else \
                return_scores[early_stop_on]
            if check_score >= self._best_score and self._save_best_model:
                eval_scores_transformed = {key:
                                               return_scores[key].item() if torch.is_tensor(return_scores[key]) else
                                               return_scores[key]
                                           for key in return_scores.keys()}
                self.save(self.output_path,
                          {"training_examples (when pos_num=1 for ranking)": len(self._train_data),
                           "evaluation_steps": self._evaluation_steps,
                           "train_batch_size": self._train_batch_size, "epoch": epoch + 1, "total_epochs": self._epochs,
                           "steps": steps,
                           "saved_at_total_steps": steps + epoch * self._steps_per_epoch,
                           "steps_per_epoch": self._steps_per_epoch, "eval_scores_on_dev": eval_scores_transformed})

                self._best_score = check_score

                logger.info(f"  Save check-point at epoch={epoch} step={steps}")
                self._early_stop_count = 0
            else:
                self._early_stop_count += 1

        return return_scores.pop("loss").item() if "loss" in return_scores else None, return_scores

    def save(self, path, eval_details):
        if path is None:
            return
        logger.info(f"   Save model to {path}")
        contained_modules = []

        to_iterate = self.model.module._modules if self._n_gpu > 1 else self.model._modules

        for idx, name in enumerate(to_iterate):
            module = to_iterate[str(name)]

            model_path = os.path.join(path, str(idx) + "_" + type(module).__name__)
            os.makedirs(model_path, exist_ok=True)
            module.save(model_path)
            contained_modules.append(
                {'idx': idx, 'name': name, 'path': os.path.basename(model_path), 'type': type(module).__module__})

        if self.wandb_config != None:
            with open(os.path.join(path, 'hyperparams.json'), 'w') as f:
                json.dump(self.wandb_config.__dict__, f, indent=2)

        with open(os.path.join(path, 'modules.json'), 'w') as fOut:
            json.dump(contained_modules, fOut, indent=2)
        with open(os.path.join(path, 'ck_report.json'), 'w') as fOut:
            json.dump(eval_details, fOut, indent=2)

    def _get_scheduler(self, optimizer, scheduler: str, warmup_steps: int, num_total: int):
        assert scheduler in ["constantlr", "warmuplinear", "warmupconstant", "warmupcosine",
                             "warmupcosinewithhardrestarts"], (
            'scheduler should be one of ["constantlr","warmupconstant","warmupcosine","warmupcosinewithhardrestarts"]')
        if scheduler == 'constantlr':
            return transformers.get_constant_schedule(optimizer)
        elif scheduler == 'warmupconstant':
            return transformers.get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
        elif scheduler == 'warmuplinear':
            return transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                                num_training_steps=num_total)
        elif scheduler == 'warmupcosine':
            return transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                                num_training_steps=num_total)
        elif scheduler == 'warmupcosinewithhardrestarts':
            return transformers.get_cosine_with_hard_restarts_schedule_with_warmup(optimizer,
                                                                                   num_warmup_steps=warmup_steps,
                                                                                   num_training_steps=num_total)

    def time_diff(self, t_a, t_b):
        t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
        return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

## 第四步：定义数据集处理模块

In [ ]:
import json
import logging
import os
import pickle

from torch.utils.data import Dataset

logger = logging.getLogger(__name__)
from tqdm import tqdm

class SrcCodeDataset(Dataset):
    def __init__(self, file_path, model, cache_path=None):
        """
        this dataset class is used to load source code dataset in batch for fine-tuning with GPT2LMModel
        :param model: the model that the dataset will be fed to
        """
        self.inputs = []
        load_cache = False
        if cache_path != None:
            load_cache = self._load_cache(cache_path)
        if not load_cache:
            self._build(file_path, model)
        if cache_path != None:
            self._cache(cache_path)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        input_ids = self.inputs[index]["input_ids"]
        # input_mask = self.inputs[index]["attention_mask"] we don't need attention_mask for this task
        # return {"input_ids": input_ids, "input_mask": input_mask}
        return {"input_ids": input_ids}

    def _load_cache(self, cache_path):
        load_cache = False
        if os.path.isdir(cache_path):
            if os.path.isfile(os.path.join(cache_path, "inputs.pk")):
                with open(os.path.join(cache_path, "inputs.pk"), "rb") as f:
                    logger.info(
                        f"  load cached token ids of model from {cache_path}")
                    self.inputs = pickle.load(f)
                    load_cache = True
        return load_cache

    def _cache(self, cache_path):
        if not os.path.isdir(cache_path):
            os.makedirs(cache_path)
        with open(os.path.join(cache_path, "inputs.pk"), "wb") as f:
            pickle.dump(self.inputs, f)
            logger.info(
                f"  save tokenized ids of samples to: {cache_path}/inputs.pk")

    def _build(self, file_path, model):
        with open(file_path) as f:
            for line in tqdm(f):
                example = json.loads(line.strip())
                if example["label"].lower() == "python":
                    encoded_plus = model.tokenizer.encode_plus(
                        model.tokenize("<python>") + example["token_ids"] + [model.eos_token_id],
                        max_length=model.max_seq_length)
                elif example["label"].lower() == "java":
                    encoded_plus = model.tokenizer.encode_plus(
                        model.tokenize("<java>") + example["token_ids"] + [model.eos_token_id],
                        max_length=model.max_seq_length)
                self.inputs.append(encoded_plus.data)

## 第五步：开始训练

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import logging
import os
import argparse

logging.basicConfig(
    format=logging.BASIC_FORMAT,
    datefmt='%Y-%m-%d %H:%M:%S',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

dic = dict(model_select='distilgpt2',
           # model_select='/kaggle/input/distilgpt2-model/distilgpt2',
           dataset_name='source_code',
           per_gpu_train_batch_size=4,
           dev_batch_size=8,

           num_epochs_train=10,
           max_seq_length=256,
           lr=2e-05,
           warmup_ratio=0.2,

           early_stop=20,
           scheduler='warmuplinear',
           seed=122,
           accumulation_steps=1,
           n_gpu=1,
           visiable_device='0',
           evaluation_steps=200,
           wandb_project_name='code_generate',
           restore_training=False,
           with_wandb=True)

args = argparse.Namespace(**dic)
logger.info(f"args: {args}")
dataset_folder = f"/content/drive/MyDrive/lab2/auto_coding-master/dataset/{args.dataset_name}/json/"
output_path = f"/content/drive/MyDrive/lab2/auto_coding-master/model/{args.model_select}_fine_tuned_coder_1"
# dataset_folder = f"/kaggle/input/distligpt2-data/source_code/json/"
# output_path = f"/kaggle/working/model/distligpt2_fine_tuned_coder_1"
# initialize model by model name (the same as used in transformers lib)
model = GPTSingleHead(args.model_select, max_seq_length=args.max_seq_length)
# add special tokens for controlling code generation by different programming language
model.add_special_words({"pad_token": "<pad>", "additional_special_tokens": ["<python>", "<java>"]})
# load training dataset
file_path = dataset_folder + "train.jsonl"
train_dataset = SrcCodeDataset(file_path, model, cache_path=os.path.join(".cache", output_path, "train"))
# load developlemt dataset
file_path = dataset_folder + "dev.jsonl"
dev_dataset = SrcCodeDataset(file_path, model, cache_path=os.path.join(".cache", output_path, "dev"))
# initialize development evaluator
dev_evaluator = SingleCLMEvaluator()
# initialize model trainer
model_trainer = ModelTrainer(model,
                             train_dataset=train_dataset,
                             dev_dataset=dev_dataset,
                             dev_evaluator=dev_evaluator,
                             scheduler=args.scheduler,
                             epochs=args.num_epochs_train,
                             per_gpu_train_batch_size=args.per_gpu_train_batch_size,
                             output_path=output_path,
                             optimizer_params={'lr': args.lr, 'eps': 1e-6, 'correct_bias': False},
                             evaluation_steps=args.evaluation_steps,
                             early_stop=args.early_stop,
                             dev_batch_size=args.dev_batch_size,
                             restore_training=args.restore_training,
                             accumulation_steps=args.accumulation_steps,
                             n_gpu=args.n_gpu,
                             visiable_device=args.visiable_device,
                             warmup_ratio=args.warmup_ratio,
                             seed=args.seed,
                             data_loader_shuffle=True,
                             wandb_config=args if args.with_wandb else None)
# start training
model_trainer.train()